<a href="https://colab.research.google.com/github/rhiga2/AIHawaii/blob/main/workshops/ai_agents/ReasoningAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reasoning Agent
## Overview
1. Set up LLM client
2. Set up custom tools
3. ReAct flow
4. Test example

## Setup LLM Client
1. Set up OpenAI key
2. Add OpenAI key to secrets

In [10]:
import os
from openai import OpenAI
from google.colab import userdata

# Initialize the OpenAI client
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
client = OpenAI()

In [11]:
completion = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello! What is the weather in Honolulu?"}
  ]
)

print(completion.choices[0].message.content)

I currently don't have access to real-time data, including weather updates. To get the current weather conditions in Honolulu, you can check a reliable weather website like the National Weather Service, Weather.com, or use a weather app on your smartphone.


## Setup Custom Tools

In [7]:
!pip install langgraph
!pip install -U langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 5.3 MB/s eta 0:00:00


In [22]:
from langgraph.prebuilt import create_react_agent
import json

recipes = {
    "shepards pie": {
        "carrots": 3,
        "peas": 1,
        "onions": 1,
        "ground beef": 1,
        "potatoes": 5,
        "heavy cream": 1
    },
    "tuna salad": {
        "tuna": 1,
        "lettuce": 1,
        "tomato": 1,
        "mayonaise": 1
    },
    "garlic bread": {
        "garlic": 1,
        "bread": 2,
    },
}

fridge_content = {
    "eggs": 2,
    "ground beef": 2,
    "peas": 5,
    "onions": 3
}

def get_fridge_content():
    """ Get contents of fridge """
    return f"Fridge content: {json.dumps(fridge_content)}"

def get_recipe_list() -> str:
    """ Gets recipe list """
    return f"Recipe list: {json.dumps(recipes)}"

def get_ingredients_to_buy(recipe_name: str):
    """ Get ingredients to buy for a recipe """
    recipe = recipes[recipe_name]
    ingredients_to_buy = {}
    for ingredient, quantity in recipe.items():
        if ingredient not in fridge_content:
            ingredients_to_buy[ingredient] = quantity
        else:
            if fridge_content[ingredient] < quantity:
                ingredients_to_buy[ingredient] = quantity - fridge_content[ingredient]
    return f"Ingredients to buy: {json.dumps(ingredients_to_buy)}"


agent = create_react_agent(
    model="openai:gpt-4-turbo",
    tools=[get_fridge_content, get_recipe_list, get_ingredients_to_buy],
    prompt="""
    You are a helpful cooking assistant. Your job is to plan and suggest
    recipes from the recipe list based on ingredients the user may already have.
    After you suggest a recipe also include ingredients to buy and which quantity.
    """
)

# Run the agent
response = agent.invoke(
    {"messages": [{"role": "user", "content": "what should I cook for dinner"}]}
)
for message in response["messages"]:
    print(message.content)

what should I cook for dinner

Fridge content: {"eggs": 2, "ground beef": 2, "peas": 5, "onions": 3}
Recipe list: {"shepards pie": {"carrots": 3, "peas": 1, "onions": 1, "ground beef": 1, "potatoes": 5, "heavy cream": 1}, "tuna salad": {"tuna": 1, "lettuce": 1, "tomato": 1, "mayonaise": 1}, "garlic bread": {"garlic": 1, "bread": 2}}
Based on the contents of your fridge and the available recipes, I recommend you make **Shepherd's Pie** for dinner. You have almost all the necessary ingredients for this recipe.

Ingredients required for Shepherd's Pie:
- Carrots: 3
- Peas: 1 
- Onions: 1 
- Ground Beef: 1 
- Potatoes: 5
- Heavy Cream: 1

You already have:
- Peas: 5 (you need just 1)
- Onions: 3 (you need just 1)
- Ground Beef: 2 (you need just 1)

Let's check which ingredients you'll need to buy to make Shepherd's Pie.
Ingredients to buy: {"carrots": 3, "potatoes": 5, "heavy cream": 1}
To complete the ingredients for Shepherd's Pie, you need to buy the following:

- Carrots: 3
- Potatoes: